# **Measure Tuned (Dynamic) DUT Using VNA Internal Cal**

In [ ]:
import sys
sys.path.append('C:/Users/smithlab/Documents/Aaron/Modules/')
import numpy as np
import numpy.matlib
from matplotlib import pyplot as plt
import VNA_control as vc
import calibration_functions as cf
import scipy.constants
import scipy.io
import time
import datetime
import mainPyDAC2_module as tune

C     = scipy.constants.c
EPS_0 = scipy.constants.epsilon_0 #C^2/(N*m^2)
MU_0  = scipy.constants.mu_0    #m kg s^-2 A^-2
mm = 0.001
cm    = 0.01
GHz   = 1.0E9

### Load tuning states

In [2]:
# mat = scipy.io.loadmat('Path/to/tuning/states')
# ts = mat['ts'].reshape((96, 50))

# single tuning state
# ts = 2.5*np.ones((240))

# sweep all elements from 0-5 V
ts = numpy.matlib.repmat(np.linspace(0, 5, 50), 240, 1)

# sweep test (161) element from 0-5 V
# ts = np.zeros((240, 50))
# ts[pm.element_pin_index[161 - 1] - 1,:] = np.linspace(0, 5, 51)

NumApMasks = ts.shape[1]

(240, 50)


### Initialize Arduino

In [ ]:
## Arduino
# BAUDRATE, PORTNAME, #DACS
numDACs = 30
tune.setSerial(115200, "COM9", numDACs)
tune.sendInputLen()
msg = tune.receiveFromArduino()

while msg.find("Received: " + str(numDACs) + " from serial.") == -1:
    msg = tune.receiveFromArduino()
    
if not (msg == "NO VAL"):
    print(msg)

### Initialize VNA

In [ ]:
fstart = 5E9
fstop = 15E9
NumFreqs = 801
IFBW = 1000
calfile = 'Aaron_21-11-05'
power = 0
savename = 'Aaron_Quickturn_Test'
sParMeas = 'S21'

f = np.linspace(fstart, fstop, NumFreqs)

vna = vc.instrument_open('TCPIP0::169.254.187.153::5025::SOCKET')
vc.VNA_initiate(vna, NumFreqs, fstart*1E-9, fstop*1E-9, IFBW, power, 'S21', calfile=calfile)

### Take measurements

In [ ]:
measurements = np.empty((NumApMasks, NumFreqs), dtype=np.complex128)

for i in range(NumApMasks):
    
    tune.testProgram30DAC(ts[:,i])
    time.sleep(0.01)

    measurements[i,:] = vc.VNA_read(vna, sParMeas)

### Plot

In [ ]:
for i in range(NumApMasks):
    plt.plot(f*1E-9, 20*np.log10(np.abs(measurements[i,:])))
plt.xlabel('f (GHz)')
plt.ylabel(r'$|S_{21}|$')

### Save

In [ ]:
data_folder = 'C:/Users/smithlab/Documents/Aaron/Data/Aaron Quick Turn Antenna/Assembled_Board_1_22-02-02/'

measurement_name = 'Test_Element_VNA_TRL'
scipy.io.savemat('{}{}_{}.mat'.format(data_folder, savename, datetime.datetime.now().strftime('%Y-%m-%d')),
                                      {'f': f,
                                       'ts': ts,
                                       'measurements': measurements})

### Clean up communications

In [ ]:
vna.close()